# Data Cleaning

### Load libraries, data

In [1]:
import os
import gc
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv('train-flattened.csv', dtype = {'fullVisitorId' : np.str})
test_df = pd.read_csv('test-flattened.csv', dtype = {'fullVisitorId' : np.str})

### The flattened train set

In [2]:
train_df.describe()

,date,visitId,visitNumber,visitStartTime,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue
count,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,871578.0,1.708337e+06,1307430.0,1.708098e+06,1.851400e+04
mean,2.017016e+07,1.498352e+09,2.335170e+00,1.498352e+09,1.0,4.429598e+00,1.0,3.696202e+00,1.251132e+08
std,6.485620e+03,1.624937e+07,9.354034e+00,1.624937e+07,0.0,8.991748e+00,0.0,6.473237e+00,4.162653e+08
min,2.016080e+07,1.470035e+09,1.000000e+00,1.470035e+09,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+04
25%,2.016122e+07,1.482738e+09,1.000000e+00,1.482738e+09,1.0,1.000000e+00,1.0,1.000000e+00,2.306750e+07
50%,2.017071e+07,1.499832e+09,1.000000e+00,1.499832e+09,1.0,1.000000e+00,1.0,1.000000e+00,4.606000e+07
75%,2.017120e+07,1.512513e+09,1.000000e+00,1.512513e+09,1.0,4.000000e+00,1.0,4.000000e+00,1.000000e+08
max,2.018043e+07,1.525158e+09,4.570000e+02,1.525158e+09,1.0,5.000000e+02,1.0,5.000000e+02,2.312950e+10


In [3]:
train_df.iloc[:5, :8]

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop


In [4]:
train_df.iloc[:5, 8:24]

,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.campaign
0,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1.0,1,1.0,1.0,NaN,NaN,(not set)
1,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,2,NaN,2.0,NaN,NaN,(not set)
2,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,2,1.0,2.0,NaN,NaN,(not set)
3,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,2,1.0,2.0,NaN,NaN,(not set)
4,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,2,1.0,2.0,NaN,NaN,(not set)


In [5]:
train_df.iloc[:5, 24:]

,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions
0,NaN,water bottle,organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]"
1,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,"[{'index': '4', 'value': 'North America'}]"
2,True,NaN,(none),NaN,(direct),"[{'index': '4', 'value': 'North America'}]"
3,NaN,(not provided),organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]"
4,NaN,(not provided),organic,NaN,google,"[{'index': '4', 'value': 'Central America'}]"


### The flattened test set

In [6]:
test_df.describe()

,date,visitId,visitNumber,visitStartTime,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue
count,4.015890e+05,4.015890e+05,401589.000000,4.015890e+05,182678.0,401589.000000,286065.0,401488.000000,4.594000e+03
mean,2.018073e+07,1.532128e+09,2.486104,1.532128e+09,1.0,5.226087,1.0,4.214542,1.215385e+08
std,1.628911e+02,4.309457e+06,10.699105,4.309457e+06,0.0,9.313409,0.0,6.505109,5.385077e+08
min,2.018050e+07,1.525157e+09,1.000000,1.525158e+09,1.0,1.000000,1.0,1.000000,9.000000e+05
25%,2.018060e+07,1.528266e+09,1.000000,1.528266e+09,1.0,1.000000,1.0,1.000000,2.371000e+07
50%,2.018072e+07,1.531978e+09,1.000000,1.531978e+09,1.0,2.000000,1.0,2.000000,4.798000e+07
75%,2.018090e+07,1.535926e+09,2.000000,1.535926e+09,1.0,6.000000,1.0,5.000000,9.594000e+07
max,2.018102e+07,1.539673e+09,523.000000,1.539673e+09,1.0,500.000000,1.0,500.000000,3.016994e+10


In [7]:
test_df.iloc[:5, :8]

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet


In [8]:
test_df.iloc[:5, 8:24]

,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.campaign
0,True,Android,(not set),Asia,India,(not set),unknown.unknown,Delhi,Southern Asia,NaN,4,NaN,3.0,NaN,(not set),(not set)
1,False,Macintosh,San Francisco,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,4,NaN,3.0,NaN,(not set),(not set)
2,False,Chrome OS,not available in demo dataset,Americas,United States,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,Northern America,NaN,4,NaN,3.0,NaN,(not set),(not set)
3,True,iOS,Houston,Americas,United States,Houston TX,(not set),Texas,Northern America,NaN,5,NaN,4.0,NaN,(not set),(not set)
4,True,Windows,Irvine,Americas,United States,Los Angeles CA,com,California,Northern America,NaN,5,1.0,4.0,NaN,(not set),(not set)


In [9]:
test_df.iloc[:5, 24:]

,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions
0,True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'APAC'}]"
1,True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
2,True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"
3,True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
4,NaN,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"


### Columns of flattened train set

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 30 columns):
channelGrouping               object
date                          int64
fullVisitorId                 object
visitId                       int64
visitNumber                   int64
visitStartTime                int64
device.browser                object
device.deviceCategory         object
device.isMobile               bool
device.operatingSystem        object
geoNetwork.city               object
geoNetwork.continent          object
geoNetwork.country            object
geoNetwork.metro              object
geoNetwork.networkDomain      object
geoNetwork.region             object
geoNetwork.subContinent       object
totals.bounces                float64
totals.hits                   int64
totals.newVisits              float64
totals.pageviews              float64
totals.transactionRevenue     float64
trafficSource.adContent       object
trafficSource.campaign        object
tr

### Columns of flattened test set

In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 30 columns):
channelGrouping               401589 non-null object
date                          401589 non-null int64
fullVisitorId                 401589 non-null object
visitId                       401589 non-null int64
visitNumber                   401589 non-null int64
visitStartTime                401589 non-null int64
device.browser                401589 non-null object
device.deviceCategory         401589 non-null object
device.isMobile               401589 non-null bool
device.operatingSystem        401589 non-null object
geoNetwork.city               401589 non-null object
geoNetwork.continent          401589 non-null object
geoNetwork.country            401589 non-null object
geoNetwork.metro              401589 non-null object
geoNetwork.networkDomain      401589 non-null object
geoNetwork.region             401589 non-null object
geoNetwork.subContinent       401589 non-null o

In [12]:
# drop constants

columns = [col for col in train_df.columns if train_df[col].nunique() > 1]

train_df = train_df[columns]
test_df = test_df[columns]

In [13]:
train_len = train_df.shape[0]

In [14]:
# Combine Dataframes for formatting
merged_df = pd.concat([train_df, test_df])

In [15]:
# Calculate total visit time 
merged_df['total_visitId_time'] = merged_df['visitId'] - merged_df['visitStartTime']
merged_df['total_visitId_time'] = (merged_df['total_visitId_time'] != 0).astype(int)
del merged_df['visitId']

In [16]:
def _add_date_features(df):
    
    df['date'] = df['date'].astype(str)
    df["date"] = df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    df["date"] = pd.to_datetime(df["date"])
    
    df["month"]   = df['date'].dt.month
    df["day"]     = df['date'].dt.day
    df["weekday"] = df['date'].dt.weekday
    return df 

merged_df = _add_date_features(merged_df)

In [17]:
merged_df.head(5)

,channelGrouping,date,fullVisitorId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,...,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions,total_visitId_time,month,day,weekday
0,Organic Search,2017-10-16,3162355547410993243,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,...,(not set),water bottle,organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]",0,10,16,0
1,Referral,2017-10-16,8934116514970143966,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,...,(not set),NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com,"[{'index': '4', 'value': 'North America'}]",0,10,16,0
2,Direct,2017-10-16,7992466427990357681,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,...,(not set),NaN,(none),NaN,(direct),"[{'index': '4', 'value': 'North America'}]",0,10,16,0
3,Organic Search,2017-10-16,9075655783635761930,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,...,(not set),(not provided),organic,NaN,google,"[{'index': '4', 'value': 'EMEA'}]",0,10,16,0
4,Organic Search,2017-10-16,6960673291025684308,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,...,(not set),(not provided),organic,NaN,google,"[{'index': '4', 'value': 'Central America'}]",0,10,16,0


In [18]:
# Split dataframes into test/train

train_df = merged_df[:train_len]
test_df = merged_df[train_len:]

In [19]:
train_df.to_csv("train-flat-clean.csv", index=False)
test_df.to_csv("test-flat-clean.csv", index=False)